<a href="https://colab.research.google.com/github/nmorado/adhoc/blob/master/fengshui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import pickle

In [0]:
import os

execution_path = os.getcwd()

from keras.preprocessing import image
import numpy as np


In [0]:
import cv2
import matplotlib.pyplot as plt

# load the data

In [0]:
_training_X = pickle.load(open("training.X.pickle", "rb"))
_training_y = pickle.load(open("training.y.pickle", "rb"))
_test_X = pickle.load(open("test.X.pickle", "rb"))
_test_y = pickle.load(open("test.y.pickle", "rb"))

In [7]:
# image normalization
_training_X = _training_X / 255.0
_test_X = _test_X / 255.0

# build convolutional network
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[500, 500, 3]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))


model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))


model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

print(model.summary())

model.compile(loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=["sparse_categorical_accuracy"])

model.fit(_training_X, _training_y, epochs=5)

test_loss, test_accuracy = model.evaluate(_test_X, _test_y)

print("Test accuracy: {}".format(test_accuracy))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 500, 500, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 500, 500, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 250, 250, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 250, 250, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 125, 125, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1000000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [0]:
model.save('fengshui.h5')

Test an image


In [55]:
CATEGORIES = ["bedroom", "entrance", "kitchen", "living", "master", "study"]
IMG_RES = (500, 500)
# ------------------------
# in sample              |
# ------------------------
# img = 'studyroom.jpg'
# img = 'livingroom.png'
img = 'entrance.jpg'
# img = 'master.jpg'
# img = 'bedroom.jpg'

# ------------------------
# not in sample          |
# ------------------------
# img = 'kitchen.png'

test_image = image.load_img(img, target_size=IMG_RES)

img_array = cv2.imread(img, cv2.IMREAD_COLOR)
new_array = cv2.resize(img_array, IMG_RES)

input = np.array(new_array).reshape(-1, IMG_RES[0], IMG_RES[1], 3)
input = input/255


result = model.predict_classes(input)
# print(CATEGORIES[result[0]])

plt.imshow(plt.imread(img))
plt.title(CATEGORIES[result[0]], color='w')
plt.show()